In [2]:
!nvidia-smi

Tue Jun  7 03:29:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##Dataset

In [1]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d daavoo/3d-mnist

 99% 151M/153M [00:01<00:00, 117MB/s]
100% 153M/153M [00:01<00:00, 117MB/s]


In [4]:
!unzip *.zip && rm -rf *.zip

Archive:  3d-mnist.zip
  inflating: full_dataset_vectors.h5  
  inflating: plot3D.py               
  inflating: test_point_clouds.h5    
  inflating: train_point_clouds.h5   
  inflating: voxelgrid.py            


## Libraries

In [16]:
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [7]:
with h5py.File('full_dataset_vectors.h5', 'r') as dataset:
  xtrain, xtest = dataset["X_train"][:], dataset["X_test"][:]
  ytrain, ytest = dataset["y_train"][:], dataset["y_test"][:]


In [9]:
xtrain = np.array(xtrain)
xtest = np.array(xtest)
ytest = np.array(ytest)
ytrain = np.array(ytrain)

In [13]:
print('train shape:', xtrain.shape)
print("test shape:", xtest.shape)

train shape: (10000, 4096)
test shape: (2000, 4096)


In [14]:
xtrain = xtrain.reshape(xtrain.shape[0], 16, 16, 16, 1)
xtest = xtest.reshape(xtest.shape[0], 16, 16, 16, 1)

In [15]:
print('train shape:', xtrain.shape)
print("test shape:", xtest.shape)

train shape: (10000, 16, 16, 16, 1)
test shape: (2000, 16, 16, 16, 1)


In [33]:
ytrain, ytest = tf.keras.utils.to_categorical(ytrain, 10), tf.keras.utils.to_categorical(ytest, 10)

In [34]:
ytrain

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

##Model

In [25]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=(16, 16, 16, 1), bias_initializer=tf.keras.initializers.Constant(0.01)))
model.add(tf.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', bias_initializer=tf.keras.initializers.Constant(0.01)))
model.add(tf.keras.layers.MaxPooling3D((2, 2, 2)))

model.add(tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu'))
model.add(tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling3D((2, 2, 2)))

model.add(tf.keras.layers.Dropout(0.6))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 14, 14, 14, 32)    896       
                                                                 
 conv3d_1 (Conv3D)           (None, 12, 12, 12, 32)    27680     
                                                                 
 max_pooling3d (MaxPooling3D  (None, 6, 6, 6, 32)      0         
 )                                                               
                                                                 
 conv3d_2 (Conv3D)           (None, 4, 4, 4, 64)       55360     
                                                                 
 conv3d_3 (Conv3D)           (None, 2, 2, 2, 64)       110656    
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 1, 1, 1, 64)      0         
 3D)                                                    

In [35]:
model.compile(tf.keras.optimizers.Adam(0.001),'categorical_crossentropy',['accuracy'])

In [36]:
history = model.fit(
    xtrain,
    ytrain,
    epochs=200,
    batch_size=32,
    verbose=1,
    validation_data=(xtest,ytest),
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=15)]
)

Epoch 1/200
313/313 [==============================] - 15s 12ms/step - loss: 2.1302 - accuracy: 0.2069 - val_loss: 1.7436 - val_accuracy: 0.3740
Epoch 2/200
313/313 [==============================] - 3s 10ms/step - loss: 1.6478 - accuracy: 0.4064 - val_loss: 1.2845 - val_accuracy: 0.5665
Epoch 3/200
313/313 [==============================] - 3s 10ms/step - loss: 1.4197 - accuracy: 0.4918 - val_loss: 1.1226 - val_accuracy: 0.6145
Epoch 4/200
313/313 [==============================] - 3s 11ms/step - loss: 1.2978 - accuracy: 0.5412 - val_loss: 1.0790 - val_accuracy: 0.6190
Epoch 5/200
313/313 [==============================] - 3s 10ms/step - loss: 1.2308 - accuracy: 0.5661 - val_loss: 1.0420 - val_accuracy: 0.6385
Epoch 6/200
313/313 [==============================] - 3s 10ms/step - loss: 1.1797 - accuracy: 0.5800 - val_loss: 1.0611 - val_accuracy: 0.6185
Epoch 7/200
313/313 [==============================] - 3s 11ms/step - loss: 1.1426 - accuracy: 0.5937 - val_loss: 0.9842 - val_accuracy

In [37]:
_, acc = model.evaluate(xtrain, ytrain)
print('training accuracy:', str(round(acc*100, 2))+'%')
_, acc = model.evaluate(xtest, ytest)
print('testing accuracy:', str(round(acc*100, 2))+'%')

313/313 [==============================] - 2s 5ms/step - loss: 0.3908 - accuracy: 0.8865
training accuracy: 88.65%
63/63 [==============================] - 0s 5ms/step - loss: 0.9250 - accuracy: 0.6965
testing accuracy: 69.65%


In [38]:
ytest[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

In [40]:
xtest[0].shape

(16, 16, 16, 1)

In [42]:
yhat = model.predict(tf.expand_dims(xtest[0], 0))

In [44]:
np.argmax(yhat), np.argmax(ytest[0])

(7, 7)

###Implemented by Muhammad Hanan Asghar